In [1]:
!pip install apyori
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import pandas as pd
from apyori import apriori
from itertools import combinations
from operator import itemgetter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

  Created wheel for apyori: filename=apyori-1.1.2-cp36-none-any.whl size=5975 sha256=91ba608c633939663455d8095f6ca98ace85278e3fe26a867f433f86784a325b
  Stored in directory: /root/.cache/pip/wheels/5d/92/bb/474bbadbc8c0062b9eb168f69982a0443263f8ab1711a8cad0
Successfully built apyori


In [2]:
%cd 'drive/My Drive/Colab Notebooks/ML TOR'
!pwd

/content/drive/My Drive/Colab Notebooks/ML TOR
/content/drive/My Drive/Colab Notebooks/ML TOR


In [3]:
df = pd.read_csv( 'MontrealData.csv',encoding='latin-1')

# New Section

In [4]:
df.head()

,CATEGORIE,DATE,QUART,PDQ,X,Y,LONGITUDE,LATITUDE
0,Vol de véhicule à moteur,2018-09-13,jour,30.0,294904.159001,5.047549e+06,-73.626778,45.567780
1,Vol de véhicule à moteur,2018-04-30,jour,30.0,294904.159001,5.047549e+06,-73.626778,45.567780
2,Vol de véhicule à moteur,2018-09-01,nuit,7.0,290274.565000,5.042150e+06,-73.685928,45.519122
3,Méfait,2017-07-21,jour,21.0,0.000000,0.000000e+00,1.000000,1.000000
4,Méfait,2017-07-29,jour,12.0,0.000000,0.000000e+00,1.000000,1.000000


In [5]:
df_pdq = pd.read_csv( 'pdq_point.csv',encoding='latin-1')
df_pdq.head()

,NO_CIV_LIE,PREFIX_TEM,NOM_TEMP,DIR_TEMP,MUN_TEMP,DESC_LIEU,Longitude,Latitude
0,4139,BOUL,SOURCES DES,NaN,DDO,POSTE DE QUARTIER 4,-73.807304,45.494380
1,8930,BOUL,PIE-IX,NaN,MTL,POSTE DE QUARTIER 30,-73.617433,45.577744
2,4555,RUE,HOCHELAGA,NaN,MTL,POSTE DE QUARTIER 23,-73.545915,45.556866
3,11756,BOUL,O'BRIEN,NaN,MTL,POSTE DE QUARTIER 10,-73.701029,45.528543
4,1435,AVEN,VAN-HORNE,NaN,OUT,POSTE DE QUARTIER 24,-73.615612,45.520110


In [6]:
df = df[df['CATEGORIE'].notna()]
df_pdq[['temp','PDQNo']] = df_pdq.DESC_LIEU.str.split('QUARTIER', expand = True)
df_pdq['PDQ'] = df_pdq['PDQNo'].astype(np.float64)
pd.to_numeric(df_pdq['PDQ'])
cols =['PDQ']
df_merge = pd.merge(df ,df_pdq,on =cols)
df_merge.head()

,CATEGORIE,DATE,QUART,PDQ,X,Y,LONGITUDE,LATITUDE,NO_CIV_LIE,PREFIX_TEM,NOM_TEMP,DIR_TEMP,MUN_TEMP,DESC_LIEU,Longitude,Latitude,temp,PDQNo
0,Vol de véhicule à moteur,2018-09-13,jour,30.0,294904.159001,5.047549e+06,-73.626778,45.567780,8930,BOUL,PIE-IX,NaN,MTL,POSTE DE QUARTIER 30,-73.617433,45.577744,POSTE DE,30
1,Vol de véhicule à moteur,2018-04-30,jour,30.0,294904.159001,5.047549e+06,-73.626778,45.567780,8930,BOUL,PIE-IX,NaN,MTL,POSTE DE QUARTIER 30,-73.617433,45.577744,POSTE DE,30
2,Introduction,2018-01-10,jour,30.0,294670.696005,5.047695e+06,-73.629772,45.569087,8930,BOUL,PIE-IX,NaN,MTL,POSTE DE QUARTIER 30,-73.617433,45.577744,POSTE DE,30
3,Méfait,2018-11-12,soir,30.0,294670.696005,5.047695e+06,-73.629772,45.569087,8930,BOUL,PIE-IX,NaN,MTL,POSTE DE QUARTIER 30,-73.617433,45.577744,POSTE DE,30
4,Méfait,2018-08-15,jour,30.0,294670.696005,5.047695e+06,-73.629772,45.569087,8930,BOUL,PIE-IX,NaN,MTL,POSTE DE QUARTIER 30,-73.617433,45.577744,POSTE DE,30


In [7]:
# Columns for the models
col_list = ['CATEGORIE','QUART','PDQ','X','Y','NO_CIV_LIE',	'PREFIX_TEM',	'NOM_TEMP', 'MUN_TEMP', 'LONGITUDE', 'LATITUDE']

# New dataframe from columns
df2 = df_merge[col_list]
df2.dropna()

#Factorize dependent variable column:
crime_var = pd.factorize(df2['CATEGORIE'])
df2['CATEGORIE'] = crime_var[0]
definition_list_CATEGORIE = crime_var[1]

QUART_var = pd.factorize(df2['QUART'])
df2['QUART'] = QUART_var[0]
definition_list_QUART = QUART_var[1] 

PDQ_var = pd.factorize(df2['PDQ'])
df2['PDQ'] = PDQ_var[0]
definition_list_PDQ = PDQ_var[1] 

X_var = pd.factorize(df2['X'])
df2['X'] = X_var[0]
definition_list_X = X_var[1] 

Y_var = pd.factorize(df2['Y'])
df2['Y'] = Y_var[0]
definition_list_Y = Y_var[1] 

NO_CIV_LIE_var = pd.factorize(df2['NO_CIV_LIE'])
df2['NO_CIV_LIE'] = NO_CIV_LIE_var[0]
definition_list_NO_CIV_LIE = NO_CIV_LIE_var[1] 

PREFIX_TEM_var = pd.factorize(df2['PREFIX_TEM'])
df2['PREFIX_TEM'] =PREFIX_TEM_var[0]
definition_list_PREFIX_TEM = PREFIX_TEM_var[1] 

NOM_TEMP_var = pd.factorize(df2['NOM_TEMP'])
df2['NOM_TEMP'] = NOM_TEMP_var[0]
definition_list_NOM_TEMP = NOM_TEMP_var[1] 

LATITUDE_var = pd.factorize(df2['LATITUDE'])
df2['LATITUDE'] = LATITUDE_var[0]
definition_list_LATITUDE = LATITUDE_var[1]

long_var = pd.factorize(df2['LONGITUDE'])
df2['LONGITUDE'] = long_var[0]
definition_list_long = long_var[1]

mun_var = pd.factorize(df2['MUN_TEMP'])
df2['MUN_TEMP'] = mun_var[0]
definition_list_mun = mun_var[1]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

In [8]:
#set X and Y:
X = df2.drop(['CATEGORIE'],axis=1).values
y = df2['CATEGORIE'].values

#split the data into train and test sets for numeric encoded dataset:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 21)


# **Random Forest**

In [9]:
#Balanced Class Weight doesn't make a big difference for results:
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 40)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test) 
y_pred

print("Accuracy of Random Forest with Balanced class weight: ",accuracy_score(y_test, y_pred))


Accuracy of Random Forest with Balanced class weight:  0.4081726960654872


# **AdaBoostClassifier**

In [10]:
from sklearn.ensemble import AdaBoostClassifier
# Create adaboost classifer object
abc = AdaBoostClassifier(n_estimators=100,learning_rate=0.001)
# Train Adaboost Classifer
model = abc.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = model.predict(X_test)
y_pred

array([1, 1, 1, ..., 1, 3, 1])

In [11]:
print("AdaBoostClassifier: ",accuracy_score(y_test, y_pred))

AdaBoostClassifier:  0.34879191972537626


# **KNeighborsClassifier**

In [12]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=4)
classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                     weights='uniform')

In [13]:
y_pred = classifier.predict(X_test)
y_pred

array([0, 1, 0, ..., 3, 3, 1])

In [14]:
from sklearn.metrics import classification_report, confusion_matrix
print("Accuracy of KNN: ",accuracy_score(y_test, y_pred))

Accuracy of KNN:  0.3852653815685239


# **LogisticRegression**

In [15]:
lr = sklearn.linear_model.LogisticRegression(multi_class='multinomial',
                                             solver='lbfgs', random_state=0)
lr.fit(X_train, y_train);
y_pred = lr.predict(X_test)
y_pred
print("Accuracy of LR: ",accuracy_score(y_test, y_pred))

Accuracy of LR:  0.34687747557433324


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
